<a href="https://www.kaggle.com/code/subarnasaikia/it-s-backpack-time-xgboost?scriptVersionId=220969613" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Importing Important libraries

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

print("Import Completed!!")

Import Completed!!


## Loading training and testing datasets

In [2]:
train_file_path = "/kaggle/input/playground-series-s5e2/train.csv"
extra_train_file_path = "/kaggle/input/playground-series-s5e2/training_extra.csv"
test_file_path = "/kaggle/input/playground-series-s5e2/test.csv"
sample_submission_file_path = "/kaggle/input/playground-series-s5e2/sample_submission.csv"

In [3]:
train_df = pd.read_csv(train_file_path)
extra_train_df = pd.read_csv(extra_train_file_path)
test_df = pd.read_csv(test_file_path)

## Analysing the data

In [4]:
extra_train_df.head(5)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,500000,Under Armour,Canvas,Small,10.0,Yes,Yes,Tote,Blue,23.882052,114.11068
1,500001,Puma,Polyester,Small,4.0,No,Yes,Backpack,Green,11.869095,129.74972
2,500002,Jansport,Polyester,Small,8.0,Yes,Yes,Tote,Red,8.092302,21.37370
3,500003,Nike,Nylon,Large,7.0,No,No,Messenger,Pink,7.719581,48.09209
4,500004,Nike,Leather,Large,9.0,No,Yes,Tote,Green,22.741826,77.32461


In [5]:
train_df.head(5)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    300000 non-null  int64  
 1   Brand                 290295 non-null  object 
 2   Material              291653 non-null  object 
 3   Size                  293405 non-null  object 
 4   Compartments          300000 non-null  float64
 5   Laptop Compartment    292556 non-null  object 
 6   Waterproof            292950 non-null  object 
 7   Style                 292030 non-null  object 
 8   Color                 290050 non-null  object 
 9   Weight Capacity (kg)  299862 non-null  float64
 10  Price                 300000 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 25.2+ MB


In [7]:
train_df.shape

(300000, 11)

In [8]:
extra_train_df.shape

(3694318, 11)

In [9]:
test_df.shape

(200000, 10)

In [10]:
extra_train_df.describe()

,id,Compartments,Weight Capacity (kg),Price
count,3.694318e+06,3.694318e+06,3.692648e+06,3.694318e+06
mean,2.347158e+06,5.434022e+00,1.800883e+01,8.135820e+01
std,1.066458e+06,2.893227e+00,6.974541e+00,3.893050e+01
min,5.000000e+05,1.000000e+00,5.000000e+00,1.500000e+01
25%,1.423579e+06,3.000000e+00,1.206854e+01,4.747778e+01
50%,2.347158e+06,5.000000e+00,1.805243e+01,8.098592e+01
75%,3.270738e+06,8.000000e+00,2.398580e+01,1.148434e+02
max,4.194317e+06,1.000000e+01,3.000000e+01,1.500000e+02


In [11]:
train_df.describe()

,id,Compartments,Weight Capacity (kg),Price
count,300000.000000,300000.000000,299862.000000,300000.000000
mean,149999.500000,5.443590,18.029994,81.411107
std,86602.684716,2.890766,6.966914,39.039340
min,0.000000,1.000000,5.000000,15.000000
25%,74999.750000,3.000000,12.097867,47.384620
50%,149999.500000,5.000000,18.068614,80.956120
75%,224999.250000,8.000000,24.002375,115.018160
max,299999.000000,10.000000,30.000000,150.000000


In [12]:
train_df.describe(include=object)

,Brand,Material,Size,Laptop Compartment,Waterproof,Style,Color
count,290295,291653,293405,292556,292950,292030,290050
unique,5,4,3,2,2,3,6
top,Adidas,Polyester,Medium,Yes,Yes,Messenger,Pink
freq,60077,79630,101906,148342,148077,100031,51690


In [13]:
extra_train_df.describe(include=object)

,Brand,Material,Size,Laptop Compartment,Waterproof,Style,Color
count,3577265,3591703,3613128,3603229,3607044,3598108,3570651
unique,5,4,3,2,2,3,6
top,Under Armour,Polyester,Medium,Yes,Yes,Messenger,Pink
freq,741043,981252,1252581,1824595,1821128,1229646,636567


In [14]:
def missing_details(df):
    print("\n------------------------------------------------")
    print("Missing Values :")
    print("------------------------------------------------")
    print( df.isnull().sum()[df.isnull().sum() > 0] )
    
    
    missing_percentage = (df.isnull().sum() / len(df)) * 100 
    print("\n------------------------------------------------")
    print("Percentage of Missing values: (%) ")
    print("------------------------------------------------")
    print(missing_percentage[missing_percentage > 0])
    

    
    total_missing_percentage = (df.isnull().sum().sum() / (df.size)) * 100
    print("\n------------------------------------------------")
    print(f"Total missing values percentage: {total_missing_percentage:.2f}%")
    print("------------------------------------------------")

In [15]:
print("\n****************************************************")
print("Missing Details of train_df: ")
print("\n****************************************************")
missing_details(train_df)


print("\n****************************************************")
print("Missing Details of extra_train_df: ")
print("\n****************************************************")
missing_details(extra_train_df)

print("\n****************************************************")
print("Missing Details of test_df: ")
print("\n****************************************************")
missing_details(test_df)


****************************************************
Missing Details of train_df: 

****************************************************

------------------------------------------------
Missing Values :
------------------------------------------------
Brand                   9705
Material                8347
Size                    6595
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
dtype: int64

------------------------------------------------
Percentage of Missing values: (%) 
------------------------------------------------
Brand                   3.235000
Material                2.782333
Size                    2.198333
Laptop Compartment      2.481333
Waterproof              2.350000
Style                   2.656667
Color                   3.316667
Weight Capacity (kg)    0.046000
dtype: float64

------------------------------------------------
Total missing values percentage: 1.73%

### One-hot encoding 
reference: [One Hot Encoding](https://www.geeksforgeeks.org/ml-one-hot-encoding/)

In [16]:
def oneHotEncdoing(df):
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    print("\n------------------------------------------------")
    print(f"categorical_columns \n{categorical_columns}")
    print("------------------------------------------------")

    df[categorical_columns] = df[categorical_columns].fillna('missing')
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    one_hot_encoded = encoder.fit_transform(df[categorical_columns])
    one_hot_df = pd.DataFrame(
        one_hot_encoded,
        columns=encoder.get_feature_names_out(categorical_columns)
    )
    df_encoded = pd.concat([df.reset_index(drop=True), one_hot_df], axis=1)
    # for column in categorical_columns:
    #     new_column = column + "_nan"
    #     if new_column in df_encoded.columns.tolist():
    #         df_encoded = df_encoded.drop(new_column, axis=1)
    df_encoded = df_encoded.drop(categorical_columns, axis=1)
    assert df_encoded.shape[0] == df.shape[0], "Row count mismatch after encoding"
    
    return df_encoded

### Missing Values
- Handling missing values with simpleImputer.

In [17]:
def imputation(X_train, X_valid, test):
    my_imputer = SimpleImputer()
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
    imputed_test = pd.DataFrame(my_imputer.transform(test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_valid.columns = X_valid.columns
    imputed_test.columns = test.columns
    
    return imputed_X_train, imputed_X_valid, imputed_test

### Training and Evaluation

In [18]:
y = extra_train_df.Price
X = extra_train_df.drop(['Price'], axis=1)

print(X.shape)
print(y.shape)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

(3694318, 10)
(3694318,)


In [19]:
print(X_train.shape)
print(y_train.shape)

(2955454, 10)
(2955454,)


In [20]:
print(X_train.shape)
new_X_train = oneHotEncdoing(X_train)
new_X_valid = oneHotEncdoing(X_valid)
new_test_df = oneHotEncdoing(test_df)
print(new_X_train.shape)

final_X_train, final_X_valid, final_test_df = imputation(new_X_train, new_X_valid, new_test_df)

(2955454, 10)

------------------------------------------------
categorical_columns 
['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
------------------------------------------------

------------------------------------------------
categorical_columns 
['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
------------------------------------------------

------------------------------------------------
categorical_columns 
['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
------------------------------------------------
(2955454, 35)


In [21]:
final_X_train.head(5)

,id,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Brand_missing,Material_Canvas,...,Style_Messenger,Style_Tote,Style_missing,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red,Color_missing
0,2164168.0,5.0,6.668811,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2791140.0,8.0,23.311326,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2780243.0,7.0,29.674513,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4173761.0,9.0,13.588812,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,854040.0,8.0,28.545519,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
print(final_X_train.shape)
print(y_train.shape)

(2955454, 35)
(2955454,)


In [23]:
print("\n****************************************************")
print("Missing Details of final_X_train: ")
print("\n****************************************************")
missing_details(final_X_train)


print("\n****************************************************")
print("Missing Details of final_X_valid: ")
print("\n****************************************************")
missing_details(final_X_valid)

print("\n****************************************************")
print("Missing Details of final_test_df: ")
print("\n****************************************************")
missing_details(final_test_df)


****************************************************
Missing Details of final_X_train: 

****************************************************

------------------------------------------------
Missing Values :
------------------------------------------------
Series([], dtype: int64)

------------------------------------------------
Percentage of Missing values: (%) 
------------------------------------------------
Series([], dtype: float64)

------------------------------------------------
Total missing values percentage: 0.00%
------------------------------------------------

****************************************************
Missing Details of final_X_valid: 

****************************************************

------------------------------------------------
Missing Values :
------------------------------------------------
Series([], dtype: int64)

------------------------------------------------
Percentage of Missing values: (%) 
------------------------------------------------

## Train the model

In [24]:
model = XGBRegressor(
    n_estimators=5000, 
    learning_rate=0.05,
    early_stopping_rounds=100
)

model.fit(final_X_train, y_train, 
         eval_set=[(final_X_valid, y_valid)], 
         verbose=True
    )

[0]	validation_0-rmse:38.91286
[1]	validation_0-rmse:38.90992
[2]	validation_0-rmse:38.90731
[3]	validation_0-rmse:38.90492
[4]	validation_0-rmse:38.90281
[5]	validation_0-rmse:38.90084
[6]	validation_0-rmse:38.89895
[7]	validation_0-rmse:38.89726
[8]	validation_0-rmse:38.89557
[9]	validation_0-rmse:38.89407
[10]	validation_0-rmse:38.89250
[11]	validation_0-rmse:38.89116
[12]	validation_0-rmse:38.88970
[13]	validation_0-rmse:38.88859
[14]	validation_0-rmse:38.88740
[15]	validation_0-rmse:38.88627
[16]	validation_0-rmse:38.88518
[17]	validation_0-rmse:38.88425
[18]	validation_0-rmse:38.88325
[19]	validation_0-rmse:38.88247
[20]	validation_0-rmse:38.88178
[21]	validation_0-rmse:38.88090
[22]	validation_0-rmse:38.88025
[23]	validation_0-rmse:38.87965
[24]	validation_0-rmse:38.87911
[25]	validation_0-rmse:38.87843
[26]	validation_0-rmse:38.87768
[27]	validation_0-rmse:38.87711
[28]	validation_0-rmse:38.87653
[29]	validation_0-rmse:38.87580
[30]	validation_0-rmse:38.87505
[31]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=100,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [25]:
pred = model.predict(final_X_valid)
mae = mean_absolute_error(y_valid, pred)
rmse = mean_squared_error(y_valid, pred, squared=False)
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

MAE: 33.58641850354456
RMSE: 38.854493170771505


In [26]:
final_prediction = model.predict(final_test_df)

In [27]:
submission = pd.DataFrame({'id': new_test_df['id'], 'Price': final_prediction})
submission.to_csv('submission.csv', index=False)